# Análisis Exploratorio

Carga inicial de librerías:

In [149]:
import pandas as pd

In [151]:
df = pd.read_csv("../../data/raw/historico_completo.csv")

df.head()

,fecha,dia,tempMedia,tempMax,horMinTempMax,tempMin,horMinTempMin,humedadMedia,humedadMax,horMinHumMax,...,velVientoMax,horMinVelMax,dirVientoVelMax,radiacion,precipitacion,bateria,fechaUtlMod,et0,provincia_id,codigoEstacion
0,2005-01-01,1,9.07,18.58,14:30,0.658,07:20,67.31,87.2,07:40,...,5.008,11:51,338.1,11.110,0.0,12.93,2005-01-02T07:34:55.000+0100,1.495588,11,1
1,2005-01-02,2,9.02,12.17,14:50,5.810,21:30,84.70,91.8,23:30,...,4.106,15:44,333.5,4.173,0.0,12.92,2005-01-03T07:37:28.000+0100,0.776324,11,1
2,2005-01-03,3,8.18,15.91,14:20,3.196,06:10,80.60,92.9,11:50,...,5.174,14:43,318.2,9.090,0.2,12.94,2005-01-04T07:34:53.000+0100,1.327342,11,1
3,2005-01-04,4,10.55,18.26,12:40,3.608,07:30,68.63,87.2,06:50,...,7.680,12:48,116.6,10.400,0.0,12.93,2005-01-05T07:34:53.000+0100,1.796508,11,1
4,2005-01-05,5,9.82,17.78,14:50,2.802,07:30,67.62,87.5,06:10,...,4.508,11:12,145.7,11.090,0.0,12.93,2005-01-06T07:35:04.000+0100,1.371563,11,1


In [152]:
df.shape

(690709, 24)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690709 entries, 0 to 690708
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fecha            690709 non-null  object 
 1   dia              690709 non-null  int64  
 2   tempMedia        689485 non-null  float64
 3   tempMax          689588 non-null  float64
 4   horMinTempMax    690709 non-null  object 
 5   tempMin          689146 non-null  float64
 6   horMinTempMin    690709 non-null  object 
 7   humedadMedia     689194 non-null  float64
 8   humedadMax       688772 non-null  float64
 9   horMinHumMax     690709 non-null  object 
 10  humedadMin       688507 non-null  float64
 11  horMinHumMin     690709 non-null  object 
 12  velViento        681821 non-null  float64
 13  dirViento        681604 non-null  float64
 14  velVientoMax     681822 non-null  float64
 15  horMinVelMax     690709 non-null  object 
 16  dirVientoVelMax  682349 non-null  floa

In [155]:
df.describe()

,dia,tempMedia,tempMax,tempMin,humedadMedia,humedadMax,humedadMin,velViento,dirViento,velVientoMax,dirVientoVelMax,radiacion,precipitacion,bateria,et0,provincia_id,codigoEstacion
count,690709.000000,689485.000000,689588.000000,689146.000000,689194.000000,688772.000000,688507.000000,681821.000000,681604.000000,681822.000000,682349.000000,689706.000000,682214.000000,688780.000000,679335.000000,690709.000000,690709.000000
mean,180.901962,17.193925,24.005895,10.947206,63.673787,85.972722,37.937697,1.546672,190.716741,6.035419,194.495092,18.041855,1.273554,12.753060,3.648371,22.393258,20.006735
std,105.144790,6.909024,7.972695,6.231762,17.815849,14.063385,18.105349,0.972314,96.962238,8.779430,95.323961,8.196402,5.748707,8.805729,2.167635,11.415548,31.936172
min,1.000000,-17.910000,-7.140000,-39.870000,0.808000,10.000000,-1.016000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6999.000000,0.042203,4.000000,1.000000
25%,90.000000,11.710000,17.590000,6.357000,50.520000,79.100000,23.470000,0.906000,101.400000,4.449000,116.300000,11.070000,0.000000,12.590000,1.675095,14.000000,5.000000
50%,179.000000,16.790000,23.330000,11.110000,65.320000,90.000000,36.040000,1.324000,213.400000,5.684000,216.700000,17.910000,0.000000,12.700000,3.324815,21.000000,8.000000
75%,272.000000,22.870000,30.350000,15.860000,77.500000,97.300000,50.130000,1.906000,268.800000,7.170000,267.500000,25.660000,0.100000,12.790000,5.444859,29.000000,13.000000
max,366.000000,37.810000,134.400000,29.670000,100.000000,153.000000,100.000000,13.580000,360.000000,6999.000000,6999.000000,61.530000,2126.000000,48.000000,22.901576,41.000000,104.000000


In [156]:
df.columns

Index(['fecha', 'dia', 'tempMedia', 'tempMax', 'horMinTempMax', 'tempMin',
       'horMinTempMin', 'humedadMedia', 'humedadMax', 'horMinHumMax',
       'humedadMin', 'horMinHumMin', 'velViento', 'dirViento', 'velVientoMax',
       'horMinVelMax', 'dirVientoVelMax', 'radiacion', 'precipitacion',
       'bateria', 'fechaUtlMod', 'et0', 'provincia_id', 'codigoEstacion'],
      dtype='object')